<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Modificado el 2025-2 por Equipo Docente IIC2233. </font>
</p>

# Tabla de contenidos

1. [*QTimer*](#QTimer)
   1. [Ejemplo-multiples-QTimer](#Ejemplo-multiples-QTimer)
   2. [`singleShot`](#singleShot)
   3. [Ejemplo Aplicado](#Ejemplo-Aplicado)
2. [*Main Window*](#Main-Window)
   1. [Vista en macOs](#Vista-en-macOS)

## QTimer

Otra herramienta que existe para generar comportamiento de concurrencia dentro de PyQt es la clase `QTimer`. Los objetos de esta clase, a diferencia de los `QThread`, no son un símil a `Timer` del módulo `threading`: `Timer` después de un tiempo específico ejecuta una subrutina una única vez, mientras que `QTimer` ejecuta una subrutina cada cierto tiempo determinado periódicamente, repitiendo la subrutina una y otra vez.

Este tipo de comportamiento es simulable utilizando `QThread`, al definir código para un *thread* de la forma:

```python
def run(self):
    while True:
        # Lo que quiero que el QThread haga en cada iteración
        time.sleep(self.tiempo)
```

... y debería obtener el mismo resultado descrito al usar `QTimer`, lo que es cierto. La desventaja de realizar esto en comparación con usar un `QTimer` es que estos últimos están construidos para efectuar este comportamiento, mientras que los `QThreads`, como los *threads* en general, están construidos para que acaben eventualmente. Luego, el implementar un *thread* usando el código anterior nos obliga a usar el método `terminate`, que se considera **mala práctica** al forzar un *thread* a terminar, en vez de que este termine por su cuenta.

Por su lado, `QTimer` provee métodos para comenzar (`start`) y detener (`stop`) la ejecución periódica de buena manera. Tras inicializar un `QTimer`, se le asigna mediante `setInterval` el tiempo en milisegundos que durará el periodo entre ejecuciones, y mediante el atributo (y señal) `timeout` se puede conectar a la subrutina que se efectuará una y otra vez: `timer.timeout.connect(subrutina)`.


En el siguiente ejemplo crearemos un reloj digital que actualizará el tiempo cada 1 segundo. Este código se encuentra en el archivo `4-bonus-qtimer-y-main-window/1_reloj_digital.py`

```python
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout
from PyQt5.QtCore import QTimer
from PyQt5.QtGui import QFont
import sys
import datetime


class RelojDigital(QWidget):
    def __init__(self) -> None:
        super().__init__()

        # Crear label encargado de mostrar la hora
        self.label_timer = QLabel()
        self.label_timer.setFont(QFont("Times", 50))

        # Crear layout vertical para nuestro label
        layout = QVBoxLayout()
        layout.addWidget(self.label_timer)
        self.setLayout(layout)

        # Crear nuestro QTimer encargado de actualizar el tiempo cada 1 segundo
        timer = QTimer(self)
        timer.timeout.connect(self.mostrar_hora)
        timer.setInterval(1000)
        timer.start()

        # Definir título y tamaño ventana
        self.setWindowTitle("Reloj Digital con QTimer")
        self.setGeometry(100, 100, 250, 100)

        # Ejecutar el método para mostrar hora por primera vez
        self.mostrar_hora()

        # Mostrar ventana
        self.show()

    def mostrar_hora(self) -> None:
        # Obtener hora actual
        hora_actual = datetime.datetime.now().time()
        # Actualizar texto del label
        self.label_timer.setText(hora_actual.strftime("%H:%M:%S %p"))


if __name__ == "__main__":

    def hook(type_, value, traceback):
        print(type_)
        print(traceback)

    sys.__excepthook__ = hook
    app = QApplication([])
    reloj = RelojDigital()
    sys.exit(app.exec())

```

### Ejemplo múltiples QTimer 

A continuación se muestra el último ejemplo de `QThread` visto en el contenido anterior, pero adaptado a `QTimer` en vez de `QThread`. Este código se encuentra en el archivo `4-bonus-qtimer-y-main-window/2_multiples_qtimers.py`

```python
import sys
from PyQt5.QtCore import pyqtSignal, QObject, QTimer
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QPushButton


class MiTimer(QObject):
    senal_actualizar = pyqtSignal(int, str)

    def __init__(self, indice: int, tiempo: float) -> None:
        super().__init__()
        self.indice = indice
        self.tiempo = tiempo
        self.indice_actual = 0
        self.timer = QTimer(self)

        # Acá se asigna el tiempo de duración del periodo entre ejecuciones
        self.timer.setInterval(int(tiempo * 1000))
        # Acá se conecta la subrutina que se ejecutará
        self.timer.timeout.connect(self.enviar_dato)

    def enviar_dato(self) -> None:
        if self.indice_actual <= 9:
            self.senal_actualizar.emit(self.indice, str(self.indice_actual))
            self.indice_actual += 1
        else:
            self.senal_actualizar.emit(self.indice, "Status: Qtimer terminado")
            self.timer.stop()

    def comenzar(self) -> None:
        self.timer.start()

    def sigue_andando(self) -> bool:
        return self.timer.isActive()


class MiVentana(QWidget):
    def __init__(self) -> None:
        super().__init__()
        self.timers = []
        self.init_gui()

    def init_gui(self) -> None:
        # Configuramos los widgets de la interfaz
        # Definimos un montón de labels que corresponderán a un Qtimer cada uno
        self.labels = {i: QLabel("Status: esperando Qtimer", self) for i in range(1, 6)}
        self.boton = QPushButton("Ejecutar Qtimers", self)
        self.boton.clicked.connect(self.ejecutar_timers)

        for i in range(1, 6):
            self.labels[i].setGeometry(10, (i - 1) * 30, 230, 30)

        self.boton.setGeometry(10, 150, 230, 30)
        # Configuramos las propiedades de la ventana.
        self.setWindowTitle("Ejemplo Qtimers")
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def ejecutar_timers(self) -> None:
        """
        Este método crea cinco timers cada vez que se presiona el botón en la
        interfaz. Los timers recibirán como argumento el índice del label
        que les corresponde y el tiempo que toman entre cada iteración.
        """
        for timer in self.timers:
            if timer.sigue_andando():
                return

        self.timers = []
        for i in range(1, 6):
            timer = MiTimer(i, i / 10)
            # Se conecta la señal emitida por el timer a un método
            # de la ventana
            timer.senal_actualizar.connect(self.actualizar_labels)
            self.timers.append(timer)
            timer.comenzar()

    def actualizar_labels(self, indice: int, texto: str) -> None:
        """
        Este método actualiza el label correspondiente según los datos
        enviados desde un timer a través del índice y aplica el texto.
        """
        self.labels[indice].setText(texto)


if __name__ == "__main__":

    def hook(type_, value, traceback):
        print(type_)
        print(traceback)

    sys.__excepthook__ = hook
    app = QApplication([])
    ventana = MiVentana()
    sys.exit(app.exec())
```

### `singleShot`

Por defecto, un `QTimer` se ejecutará periódicamente cada vez que se cumpla el tiempo indicado en `setInterval`. No obstante, puede ocurrir una situación donde queremos que el `QTimer` ejecute su función una **única vez** después de transcurrir el tiempo esperado. 

Un ejemplo podría ser donde queremos que la ventana se cierre después de 10 segundos. Para modelar este ejemplo con `QTimer`, podríamos hacer:
- Un `QTimer` que se ejecuta cada 1 segundos y posee un contador. Cuando este contador llega a 10, se cierra la ventana.
- Un `QTimer` que se ejecuta cada 10 segundos. Una vez ejecutada su función, el mismo hace `stop` para no volver a ejecutarse. Además, aprovecha de cerrar la ventana.


No obstante, también existe otra forma más que es utilizar `singleShot`. Este método nos permite indicar a `pyqt` que el `QTimer` solo se ejecutará 1 vez y luego debe ser detenido. De este modo no necesitamos incluir un `stop()`.

A continuación vamos a tomar el ejemplo del `RelojAnalogico` pero vamos a incluir 2 `QTimer` que son `singleShot`. El primero, va a esperar 3 segundos y hará `hide` de la ventana, mientras que el segundo esperará 5 segundos y hará `show` de la ventana. Como son `singleShot`, solo se van a ejecutar una única vez y luego no se volverán a ejecutar.

Este código se encuentra en el archivo `4-bonus-qtimer-y-main-window/3_singleshot.py`

```python
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout
from PyQt5.QtCore import QTimer
from PyQt5.QtGui import QFont
import sys
import datetime


class RelojDigital(QWidget):
    def __init__(self) -> None:
        super().__init__()

        # Crear label encargado de mostrar la hora
        self.label_timer = QLabel()
        self.label_timer.setFont(QFont("Times", 50))

        # Crear layout vertical para nuestro label
        layout = QVBoxLayout()
        layout.addWidget(self.label_timer)
        self.setLayout(layout)

        # Crear nuestro QTimer encargado de actualizar el tiempo cada 1 segundo
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.mostrar_hora)
        self.timer.setInterval(1000)
        self.timer.start()

        # Definir título y tamaño ventana
        self.setWindowTitle("Reloj Digital con QTimer")
        self.setGeometry(100, 100, 250, 100)

        # Ejecutar el método para mostrar hora por primera vez
        self.mostrar_hora()

        # Mostrar ventana
        self.show()

        # Creamos un QTimer que despues de 3 segundos va a esconder la ventana
        self.timer_singleshot_hide = QTimer(self)
        # self.timer_singleshot_hide.setSingleShot(True)
        self.timer_singleshot_hide.timeout.connect(self.hide)
        self.timer_singleshot_hide.setInterval(3000)
        self.timer_singleshot_hide.start()

        # Creamos otro QTimer que despues de 5 segundos va a mostrar la ventana
        self.timer_singleshot_show = QTimer(self)
        # self.timer_singleshot_show.setSingleShot(True)
        self.timer_singleshot_show.timeout.connect(self.show)
        self.timer_singleshot_show.setInterval(5000)
        self.timer_singleshot_show.start()

    def mostrar_hora(self) -> None:
        # Obtener hora actual
        hora_actual = datetime.datetime.now().time()
        # Actualizar texto del label
        self.label_timer.setText(hora_actual.strftime("%H:%M:%S %p"))


if __name__ == "__main__":

    def hook(type_, value, traceback):
        print(type_)
        print(traceback)

    sys.__excepthook__ = hook
    app = QApplication([])
    reloj = RelojDigital()
    sys.exit(app.exec())
```

Mini ejercicio: Ejecuta este código una vez para ver qué sucede. Luego comenta las líneas que hacen `setSingleShot` para que ambos `QTimer` no sean `singleShot` y vuelve a ejecutar el código.  ¿Por qué crees que pasa esto? 


**Respuesta**: por defecto todo `QTimer` no es `singleShot`, así que cada 3 segundos se hará `hide` de la ventana mientras que cada 5 segundos se hará `show` de la ventana. Esto continuará hasta que se cierre la aplicación.

### Ejemplo Aplicado

Ahora, vamos a realizar el mismo ejemplo del contenido anterior pero aplicando `QTimer`.

![](img/ejemplo_aplicado.gif)

En este ejemplo, tendremos diferentes cuadrados que van de un extremo a otro a diferente velocidad. Además, se puede hacer click en un cuadrado para cambiar su color y detener su movimiento. (Mini desafío: intenta detener todos los cuadrados).

Para esto, dentro del directorio `scripts` hay un directorio llamado `ejemplo_aplicado`. En este tenemos los siguientes archivos:

* `main.py`: es el archivo principal a ejecutar.
* `parametros_general.py`: son constantes que se ocuparán tanto en _back-end_ y _front-end_.

**backend**
* `backend/logica_qthread.py`: es el archivo principal del _back-end_ donde se aplica toda la lógica del movimiento de los cuadrados. En este archivo se ocupan `QThread` para modelar cada cuadrado. Se recomienda ignorar este archivo para este ejemplo. Se utilizó en el contenido número 1.
* `backend/logica_thread.py`: es el archivo principal del _back-end_ donde se aplica toda la lógica del movimiento de los cuadrados. En este archivo se ocupan `threading.Thread` para modelar cada cuadrado. La única diferencia con `logica_qthread` es la herencia y que se debe explicitar que cada cuadrado es un _daemon_. Se recomienda ignorar este archivo para este ejemplo. Se utilizó en el contenido número 1.
* `backend/bonus_logica_qtimer.py`: es el mismo archivo que `logica_qthread.py` pero se utilizan `QTimer` en vez de `QThread`. **Este archivo se utilizará en este ejemplo**.
* `backend/parametros_backend.py`: son constantes que se ocuparán únicamente en _back-end_.

**frontend**
* `frontend/ventana.py`: es el archivo principal del _front-end_ donde se crea la ventana y todo elemento visual.
* `frontend/parametros_backend.py`: son constantes que se ocuparán únicamente en _front-end_.

Para la correcta ejecución de este archivo, debes abrir tu terminal, desplazarte con `cd` hasta el directorio `scripts/ejemplo_aplicado/` y luego escribir `python3 main.py`.

**Importante** para utilizar `backend/logica_qtimer.py`, debes:
1. Ir a `main.py`
2. Comentar la línea 5: `from backend.logica_qthread import Juego`
3. Descomentar la línea 13: `from backend.bonus_logica_qtimer import Juego`

Para esto vamos a utilizar el _back-end_ que utiliza `QTimer`. Podrás notar que el _front-end_ es el mismo, puesto que una correcta modularización nos permite reutilizar el mismo _front-end_, modificar el _back-end_ y que todo funcione correctamente.

## *Main Window*

Las ventanas creadas mediante `QWidget` que hemos usado hasta ahora, son ventanas simples donde pueden ser ubicados otros *widgets*. PyQt ofrece un tipo de ventana más completa denominada `MainWindow`. Esta ventana permite crear el esqueleto clásico de una aplicación como la mostrada en la figura a continuación, con barra de estado, barra de herramientas y barra de menú.

![](img/pyqt-mainwindow-layout.png)

La **barra de estado** permite mostrar información del estado de la aplicación en la medida que el usuario interactúa con ella. Para crearla usamos el método `statusBar()` perteneciente a la clase `QApplication`.

La **barra de menú** es una de las partes típicas de una GUI. Esta barra corresponde a un grupo de comandos organizados y agrupados de manera lógica en menús.

La **barra de herramientas** provee un acceso rápido a la mayoría de los comandos usados frecuentemente, la que puedes mover en la misma interfaz.

Finalmente, el contenido central o ***central widget*** corresponde al cuerpo de la ventana. Este *widget* puede contener cualquiera de los *widgets* en `QtWidgets`, como también alguna de las ventanas creadas en los ejemplos anteriores. Para agregar cualquier *widget* o formulario al *widget* central se utiliza el método `setCentralWidget(widget)`. 

El siguiente ejemplo muestra cómo integrar los elementos descritos en la ventana principal. Este código se encuentra en el archivo `4-bonus-qtimer-y-main-window/4_mainwindows.py`. Dado el uso de _paths_ relativos, la terminal **debe** estar posicionada dentro del directorio `scripts/4-bonus-qtimer-y-main-window` para hacer `python3 4_mainwindows.py`.

```python
import os
import sys

from PyQt5.QtCore import pyqtSignal
from PyQt5.QtGui import QIcon
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget
from PyQt5.QtWidgets import QPushButton, QLabel, QLineEdit, QAction


class MiVentana(QWidget):
    def __init__(self, status_bar_signal: pyqtSignal) -> None:
        super().__init__()
        self.status_bar_signal = status_bar_signal
        self.init_gui()

    def init_gui(self) -> None:
        """
        Este método inicializa el main widget y sus elementos.
        """
        self.label_text = QLabel("Texto", self)
        self.print_label = QLabel("Print texto:", self)
        self.line_edit = QLineEdit("", self)

        self.boton = QPushButton("&Procesar", self)
        self.boton.resize(self.boton.sizeHint())
        self.boton.clicked.connect(self.boton_callback)

        self.label_text.move(20, 15)
        self.line_edit.setGeometry(55, 15, 100, 20)
        self.print_label.move(20, 50)
        self.boton.move(20, 80)

    def boton_callback(self) -> None:
        """
        Este método es el encargado ejecutar una acción cada vez que el botón
        es presionado. En esta caso, realiza el cambio en print_label y el status bar
        mediate la emisión de una señal en la cual se envía el texto correspondiente.
        """
        self.print_label.setText(f"Print texto: {self.line_edit.text()}")
        self.print_label.resize(self.print_label.sizeHint())
        self.status_bar_signal.emit(f"QEdit: {self.line_edit.text()}")


class MainWindow(QMainWindow):
    # Esta señal permite comunicar la barra de estados con el resto de los widgets
    # en el formulario, incluidos el central widget.
    onchange_status_bar = pyqtSignal(str)

    def __init__(self) -> None:
        super().__init__()

        """Configuramos la geometría de la ventana."""
        self.setWindowTitle("Ventana con Boton")
        self.setGeometry(200, 100, 300, 250)

        """Configuramos las acciones."""
        ver_status = QAction(QIcon(None), "&Cambiar Status", self)
        ver_status.setStatusTip("Este es un ítem de prueba")
        ver_status.triggered.connect(self.cambiar_status_bar)

        limpiar_status = QAction(QIcon(None), "&Limpiar Status", self)
        limpiar_status.setStatusTip("Esta acción limpia la barra de estado")
        limpiar_status.triggered.connect(self.limpiar_status_bar)

        buscar = QAction(QIcon(os.path.join("img", "search_icon.png")), "&Search", self)
        buscar.setStatusTip("Un ícono de búsqueda")

        salir = QAction(QIcon(None), "&Exit", self)
        salir.setShortcut("Ctrl+Q")
        salir.setStatusTip("Salir de la aplicación")
        salir.triggered.connect(QApplication.quit)

        """Creamos la barra de menú."""
        menubar = self.menuBar()
        archivo_menu = menubar.addMenu("Archivo")  # primer menú
        archivo_menu.addAction(ver_status)
        archivo_menu.addAction(salir)

        otro_menu = menubar.addMenu("Otro Menú")  # segundo menú
        otro_menu.addAction(limpiar_status)

        """Creamos la barra de herramientas."""
        toolbar = self.addToolBar("Toolbar")
        toolbar.addAction(buscar)
        toolbar.addAction(salir)

        """Incluimos la barra de estado."""
        self.statusBar().showMessage("Listo")
        self.onchange_status_bar.connect(self.actualizar_status_bar)

        """
        Configuramos el widget central con una instancia de la clase
        MiVentana(). Además cargamos la señal en el central widget para
        que este pueda interactuar con la barra de estados de la ventana
        principal.
        """
        self.form = MiVentana(self.onchange_status_bar)
        self.setCentralWidget(self.form)

    def cambiar_status_bar(self) -> None:
        self.statusBar().showMessage("Cambié el Status")

    def limpiar_status_bar(self) -> None:
        self.statusBar().showMessage("Status limpio.")

    def actualizar_status_bar(self, msg: str) -> None:
        self.statusBar().showMessage(f"Actualizado. {msg}")


if __name__ == "__main__":

    def hook(type, value, traceback):
        print(type)
        print(traceback)

    sys.__excepthook__ = hook

    app = QApplication([])
    ventana = MainWindow()
    ventana.show()
    sys.exit(app.exec())
```

### Vista en macOS

Cuando trabajamos con `QMainWindows` en macOS, la barra del menú no está junto a nuestra ventana, sino que está en una barra externa. Esta barra está en la parte superior de nuestra pantalla junto a al logo de Apple. A continuación se muestra una imagen de la ventana creada en el ejemplo anterior y donde está la barra del menú.

![](img/pyqt-mainwindow-mac.png)